In [1]:
import torch
import numpy as np
from collections import defaultdict

import pandas as pd
import pathlib
from rich import print

import matplotlib.pyplot as plt

# set pyplot theme to ggplot2
plt.style.use("ggplot")
import wandb

In [2]:
source_csv_filepath = (
    "03032024-full.csv"
)
source_csv_filepath = pathlib.Path(source_csv_filepath)

In [3]:
df = pd.read_csv(source_csv_filepath)

# Concatenating the header with the first row
new_headers = [f"{col.split('.')[0]}.{df.iloc[0][idx]}" for idx, col in enumerate(df.columns)]

# Setting the new concatenated values as column names
df.columns = new_headers

# Removing the first row from the DataFrame
df = df.drop(df.index[0])

# Resetting the DataFrame index
df.reset_index(drop=True, inplace=True)

# visualize using tabulate for ipython noteboooks
from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

/tmp/ipykernel_8429/412446167.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_headers = [f"{col.split('.')[0]}.{df.iloc[0][idx]}" for idx, col in enumerate(df.columns)]


,Model.Metric,acdc.dice-loss,ade20k.mIoU,ade20k.dice-loss,aircraft.acc@1,chexpert.auc,cifar100.loss,cifar100.acc@5,cifar100.acc@1,cityscapes.mIoU,cityscapes.dice-loss,clevr.acc@1,clevr-math.acc@5,clevr-math.acc@1,coco-10k.mIoU,coco-10k.dice-loss,coco-164k.mIoU,coco-164k.dice-loss,cubirds.acc@1,diabetic.auc,dtextures.acc@1,flickr30k.text-to-image-acc@5,flickr30k.text-to-image-acc@1,flickr30k.image-to-text-acc@1,flickr30k.image-to-text-acc@5,food101.loss,food101.acc@5,food101.acc@1,fungi.acc@1,ham10k.auc,hmdb51.acc@5,hmdb51.acc@1,imagenet1k.loss,imagenet1k.acc@5,imagenet1k.acc@1,iwildcam.mae,kinetics.acc@5,kinetics.acc@1,mini.acc@1,newyorkercaptioncontest.text-to-image-acc@5,newyorkercaptioncontest.text-to-image-acc@1,newyorkercaptioncontest.image-to-text-acc@1,newyorkercaptioncontest.image-to-text-acc@5,nyu.mIoU,nyu.dice-loss,omniglot.acc@1,pascal.mIoU,pascal.dice-loss,places365.loss,places365.acc@5,places365.acc@1,pokemonblipcaptions.text-to-image-acc@5,pokemonblipcaptions.text-to-image-acc@1,pokemonblipcaptions.image-to-text-acc@1,pokemonblipcaptions.image-to-text-acc@5,ucf.acc@5,ucf.acc@1,vgg.acc@1,winoground.text-to-image-acc@1,winoground.image-to-text-acc@1
0,AR-ViT-B16-224,0.571,17.353,0.777,0.944,0.813,1.636,83.629,55.742,0.521,0.230,52.828,98.850,58.849,22.783,0.973,21.321,0.970,0.944,0.839,0.799,0.188,0.053,0.055,0.187,0.582,96.630,83.834,0.775,0.985,57.917,28.385,0.967,93.007,78.199,4.093,63.288,36.718,0.987,0.195,0.052,0.054,0.187,8.345,0.271,0.986,15.460,0.781,2.062,78.706,46.641,0.821,0.640,0.628,0.794,84.369,65.374,0.955,0.569,0.611
1,BART,0.747,0.380,0.668,0.530,0.637,3.985,29.170,8.967,0.000,0.217,42.563,96.450,41.761,0.056,0.959,0.110,0.965,0.503,0.551,0.354,0.079,0.016,0.015,0.078,3.889,31.844,11.504,0.337,0.859,9.961,1.953,6.622,3.107,0.713,4.489,1.277,0.256,0.373,0.081,0.016,0.016,0.081,4.893,0.240,0.841,1.002,0.556,5.248,11.058,2.953,0.748,0.556,0.556,0.694,5.444,1.031,0.594,0.477,0.488
2,BERT,0.959,0.360,0.670,0.590,0.662,3.685,38.137,14.520,0.000,0.218,42.593,97.400,44.265,0.055,0.962,0.096,0.966,0.474,0.576,0.376,0.081,0.016,0.015,0.083,3.499,42.442,18.191,0.328,0.911,9.766,1.953,6.473,4.616,1.173,4.576,1.269,0.233,0.396,0.082,0.015,0.016,0.085,5.503,0.241,0.902,0.961,0.545,5.008,16.037,4.795,0.756,0.599,0.587,0.722,6.040,1.287,0.594,0.504,0.503
3,CLIP-B16-224,0.538,33.804,0.687,0.965,0.816,0.908,93.760,74.731,1.054,0.213,52.622,99.113,61.218,29.609,0.968,30.369,0.954,0.979,0.867,0.843,0.209,0.055,0.062,0.206,0.306,98.679,91.496,0.854,0.986,70.039,40.404,0.752,95.708,81.815,3.613,71.240,44.574,0.962,0.210,0.057,0.060,0.199,7.545,0.287,0.989,26.619,0.632,1.757,83.864,53.445,0.852,0.679,0.618,0.817,88.238,71.440,0.989,0.540,0.486
4,ConvNextV2-Base,0.636,37.253,0.472,0.967,0.818,0.640,97.422,84.126,2.533,0.182,52.606,99.787,77.402,26.840,0.147,25.059,0.169,0.980,0.875,0.850,0.212,0.055,0.060,0.205,0.272,98.991,92.896,0.857,0.991,77.539,50.326,0.635,96.851,85.288,3.446,75.765,49.011,0.958,0.214,0.058,0.063,0.210,5.260,0.225,0.986,32.665,0.297,1.659,85.325,54.726,0.803,0.610,0.616,0.783,94.737,84.106,0.996,0.478,0.398
5,DINO-B16-224,0.515,18.610,0.733,0.956,0.819,1.200,89.590,66.282,1.714,0.215,52.846,99.188,61.001,24.968,0.969,24.292,0.951,0.959,0.847,0.791,0.196,0.046,0.054,0.186,0.478,97.449,86.519,0.812,0.987,56.525,27.995,1.376,88.493,67.529,4.010,55.634,29.529,0.899,0.200,0.052,0.053,0.190,6.138,0.286,0.987,18.184,0.698,1.987,79.704,47.454,0.856,0.682,0.655,0.818,77.125,52.769,0.959,0.483,0.495
6,DeiT3-B16-224,0.591,21.900,0.703,0.953,0.827,1.162,90.844,66.750,2.212,0.217,52.342,99.350,62.701,28.345,0.953,26.125,0.959,0.962,0.818,0.819,0.200,0.050,0.056,0.192,0.447,97.718,87.331,0.805,0.985,64.338,36.523,0.794,94.675,82.011,3.194,66.558,39.528,0.988,0.219,0.060,0.065,0.205,7.488,0.292,0.986,22.291,0.667,1.943,80.614,48.677,0.826,0.670,0.639,0.810,88.506,70.411,0.962,0.492,0.424
7,EffFormer-s0,0.730,9.801,0.662,0.775,0.800,3.478,43.571,16.511,2.492,0.211,45.115,98.700,53.862,13.995,0.903,13.549,0.865,0.8

In [4]:
def move_column(df, column_name, position):
    """
    Move a specific column to a specified position in a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.
    column_name (str): The name of the column to move.
    position (int): The position to move the column to.

    Returns:
    pandas.DataFrame: The modified DataFrame.
    """
    # Ensure the position is within the valid range
    if position < 0 or position > len(df.columns):
        raise ValueError(f"Position must be between 0 and {len(df.columns)}")

    # Get a list of the column names
    cols = df.columns.tolist()

    # Check if the column name is in the list
    if column_name in cols:
        # Remove the column from its current position
        cols.remove(column_name)

        # Insert the column at the specified position
        cols.insert(position, column_name)

        # Reorder the DataFrame
        df = df[cols]

    return df

df = move_column(df, 'imagenet1k.acc@5', 1)  # Move 'hellobro' column to the second position
df = move_column(df, 'imagenet1k.acc@1', 2)
df = move_column(df, 'imagenet1k.loss', 3)

from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )


,Model.Metric,imagenet1k.acc@5,imagenet1k.acc@1,imagenet1k.loss,acdc.dice-loss,ade20k.mIoU,ade20k.dice-loss,aircraft.acc@1,chexpert.auc,cifar100.loss,cifar100.acc@5,cifar100.acc@1,cityscapes.mIoU,cityscapes.dice-loss,clevr.acc@1,clevr-math.acc@5,clevr-math.acc@1,coco-10k.mIoU,coco-10k.dice-loss,coco-164k.mIoU,coco-164k.dice-loss,cubirds.acc@1,diabetic.auc,dtextures.acc@1,flickr30k.text-to-image-acc@5,flickr30k.text-to-image-acc@1,flickr30k.image-to-text-acc@1,flickr30k.image-to-text-acc@5,food101.loss,food101.acc@5,food101.acc@1,fungi.acc@1,ham10k.auc,hmdb51.acc@5,hmdb51.acc@1,iwildcam.mae,kinetics.acc@5,kinetics.acc@1,mini.acc@1,newyorkercaptioncontest.text-to-image-acc@5,newyorkercaptioncontest.text-to-image-acc@1,newyorkercaptioncontest.image-to-text-acc@1,newyorkercaptioncontest.image-to-text-acc@5,nyu.mIoU,nyu.dice-loss,omniglot.acc@1,pascal.mIoU,pascal.dice-loss,places365.loss,places365.acc@5,places365.acc@1,pokemonblipcaptions.text-to-image-acc@5,pokemonblipcaptions.text-to-image-acc@1,pokemonblipcaptions.image-to-text-acc@1,pokemonblipcaptions.image-to-text-acc@5,ucf.acc@5,ucf.acc@1,vgg.acc@1,winoground.text-to-image-acc@1,winoground.image-to-text-acc@1
0,AR-ViT-B16-224,93.007,78.199,0.967,0.571,17.353,0.777,0.944,0.813,1.636,83.629,55.742,0.521,0.230,52.828,98.850,58.849,22.783,0.973,21.321,0.970,0.944,0.839,0.799,0.188,0.053,0.055,0.187,0.582,96.630,83.834,0.775,0.985,57.917,28.385,4.093,63.288,36.718,0.987,0.195,0.052,0.054,0.187,8.345,0.271,0.986,15.460,0.781,2.062,78.706,46.641,0.821,0.640,0.628,0.794,84.369,65.374,0.955,0.569,0.611
1,BART,3.107,0.713,6.622,0.747,0.380,0.668,0.530,0.637,3.985,29.170,8.967,0.000,0.217,42.563,96.450,41.761,0.056,0.959,0.110,0.965,0.503,0.551,0.354,0.079,0.016,0.015,0.078,3.889,31.844,11.504,0.337,0.859,9.961,1.953,4.489,1.277,0.256,0.373,0.081,0.016,0.016,0.081,4.893,0.240,0.841,1.002,0.556,5.248,11.058,2.953,0.748,0.556,0.556,0.694,5.444,1.031,0.594,0.477,0.488
2,BERT,4.616,1.173,6.473,0.959,0.360,0.670,0.590,0.662,3.685,38.137,14.520,0.000,0.218,42.593,97.400,44.265,0.055,0.962,0.096,0.966,0.474,0.576,0.376,0.081,0.016,0.015,0.083,3.499,42.442,18.191,0.328,0.911,9.766,1.953,4.576,1.269,0.233,0.396,0.082,0.015,0.016,0.085,5.503,0.241,0.902,0.961,0.545,5.008,16.037,4.795,0.756,0.599,0.587,0.722,6.040,1.287,0.594,0.504,0.503
3,CLIP-B16-224,95.708,81.815,0.752,0.538,33.804,0.687,0.965,0.816,0.908,93.760,74.731,1.054,0.213,52.622,99.113,61.218,29.609,0.968,30.369,0.954,0.979,0.867,0.843,0.209,0.055,0.062,0.206,0.306,98.679,91.496,0.854,0.986,70.039,40.404,3.613,71.240,44.574,0.962,0.210,0.057,0.060,0.199,7.545,0.287,0.989,26.619,0.632,1.757,83.864,53.445,0.852,0.679,0.618,0.817,88.238,71.440,0.989,0.540,0.486
4,ConvNextV2-Base,96.851,85.288,0.635,0.636,37.253,0.472,0.967,0.818,0.640,97.422,84.126,2.533,0.182,52.606,99.787,77.402,26.840,0.147,25.059,0.169,0.980,0.875,0.850,0.212,0.055,0.060,0.205,0.272,98.991,92.896,0.857,0.991,77.539,50.326,3.446,75.765,49.011,0.958,0.214,0.058,0.063,0.210,5.260,0.225,0.986,32.665,0.297,1.659,85.325,54.726,0.803,0.610,0.616,0.783,94.737,84.106,0.996,0.478,0.398
5,DINO-B16-224,88.493,67.529,1.376,0.515,18.610,0.733,0.956,0.819,1.200,89.590,66.282,1.714,0.215,52.846,99.188,61.001,24.968,0.969,24.292,0.951,0.959,0.847,0.791,0.196,0.046,0.054,0.186,0.478,97.449,86.519,0.812,0.987,56.525,27.995,4.010,55.634,29.529,0.899,0.200,0.052,0.053,0.190,6.138,0.286,0.987,18.184,0.698,1.987,79.704,47.454,0.856,0.682,0.655,0.818,77.125,52.769,0.959,0.483,0.495
6,DeiT3-B16-224,94.675,82.011,0.794,0.591,21.900,0.703,0.953,0.827,1.162,90.844,66.750,2.212,0.217,52.342,99.350,62.701,28.345,0.953,26.125,0.959,0.962,0.818,0.819,0.200,0.050,0.056,0.192,0.447,97.718,87.331,0.805,0.985,64.338,36.523,3.194,66.558,39.528,0.988,0.219,0.060,0.065,0.205,7.488,0.292,0.986,22.291,0.667,1.943,80.614,48.677,0.826,0.670,0.639,0.810,88.506,70.411,0.962,0.492,0.424
7,EffFormer-s0,72.456,44.615,2.830,0.730,9.801,0.662,0.775,0.800,3.478,43.571,16.511,2.492,0.211,45.115,98.700,53.862,13.995,0.

In [5]:
def rank_columns(df, keywords):
    """
    Rank the columns of a DataFrame based on keywords.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.
    keywords (dict): A dictionary mapping keywords to a boolean indicating whether to rank in
                     ascending order. For example, {'loss': True, 'acc': False}.

    Returns:
    pandas.DataFrame: The DataFrame with ranked columns.
    """
    for column in df.columns:
        for keyword, ascending in keywords.items():
            if keyword.lower() in column.lower():
                df[column] = df[column].rank(method='min', ascending=ascending)
                break  # Break the inner loop if keyword is found

    return df

# Define the keywords to look for in column names, and whether to rank in ascending order
keywords = {
    'loss': True,
    'acc': False,
    "miou": False,
    "auc": False,
    "mae": True,
}

df = rank_columns(df, keywords)



In [6]:
from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

,Model.Metric,imagenet1k.acc@5,imagenet1k.acc@1,imagenet1k.loss,acdc.dice-loss,ade20k.mIoU,ade20k.dice-loss,aircraft.acc@1,chexpert.auc,cifar100.loss,cifar100.acc@5,cifar100.acc@1,cityscapes.mIoU,cityscapes.dice-loss,clevr.acc@1,clevr-math.acc@5,clevr-math.acc@1,coco-10k.mIoU,coco-10k.dice-loss,coco-164k.mIoU,coco-164k.dice-loss,cubirds.acc@1,diabetic.auc,dtextures.acc@1,flickr30k.text-to-image-acc@5,flickr30k.text-to-image-acc@1,flickr30k.image-to-text-acc@1,flickr30k.image-to-text-acc@5,food101.loss,food101.acc@5,food101.acc@1,fungi.acc@1,ham10k.auc,hmdb51.acc@5,hmdb51.acc@1,iwildcam.mae,kinetics.acc@5,kinetics.acc@1,mini.acc@1,newyorkercaptioncontest.text-to-image-acc@5,newyorkercaptioncontest.text-to-image-acc@1,newyorkercaptioncontest.image-to-text-acc@1,newyorkercaptioncontest.image-to-text-acc@5,nyu.mIoU,nyu.dice-loss,omniglot.acc@1,pascal.mIoU,pascal.dice-loss,places365.loss,places365.acc@5,places365.acc@1,pokemonblipcaptions.text-to-image-acc@5,pokemonblipcaptions.text-to-image-acc@1,pokemonblipcaptions.image-to-text-acc@1,pokemonblipcaptions.image-to-text-acc@5,ucf.acc@5,ucf.acc@1,vgg.acc@1,winoground.text-to-image-acc@1,winoground.image-to-text-acc@1
0,AR-ViT-B16-224,8.000000,5.000000,5.000000,7.000000,11.000000,15.000000,8.000000,10.000000,10.000000,10.000000,11.000000,11.000000,16.000000,2.000000,8.000000,8.000000,9.000000,16.000000,9.000000,12.000000,8.000000,9.000000,6.000000,8.000000,4.000000,7.000000,7.000000,8.000000,10.000000,10.000000,9.000000,6.000000,11.000000,7.000000,13.000000,8.000000,7.000000,3.000000,8.000000,7.000000,6.000000,8.000000,1.000000,11.000000,6.000000,9.000000,16.000000,10.000000,10.000000,10.000000,6.000000,6.000000,5.000000,6.000000,5.000000,6.000000,8.000000,1.000000,1.000000
1,BART,15.000000,15.000000,15.000000,14.000000,13.000000,3.000000,15.000000,15.000000,15.000000,16.000000,2.000000,13.000000,9.000000,13.000000,15.000000,15.000000,13.000000,7.000000,13.000000,9.000000,13.000000,15.000000,15.000000,15.000000,14.000000,15.000000,16.000000,15.000000,15.000000,16.000000,13.000000,14.000000,1.000000,12.000000,14.000000,13.000000,13.000000,14.000000,15.000000,14.000000,14.000000,15.000000,14.000000,4.000000,15.000000,13.000000,4.000000,15.000000,15.000000,15.000000,13.000000,13.000000,13.000000,12.000000,13.000000,15.000000,14.000000,14.000000,9.000000
2,BERT,14.000000,14.000000,14.000000,16.000000,15.000000,4.000000,13.000000,13.000000,13.000000,14.000000,14.000000,13.000000,11.000000,12.000000,13.000000,12.000000,15.000000,8.000000,15.000000,10.000000,15.000000,13.000000,14.000000,14.000000,14.000000,15.000000,14.000000,13.000000,13.000000,14.000000,14.000000,12.000000,3.000000,12.000000,15.000000,15.000000,16.000000,13.000000,14.000000,16.000000,14.000000,14.000000,10.000000,5.000000,14.000000,14.000000,3.000000,13.000000,13.000000,12.000000,11.000000,12.000000,10.000000,11.000000,11.000000,13.000000,14.000000,7.000000,5.000000
3,CLIP-B16-224,2.000000,4.000000,2.000000,4.000000,3.000000,10.000000,3.000000,8.000000,5.000000,5.000000,6.000000,9.000000,7.000000,3.000000,5.000000,4.000000,4.000000,11.000000,2.000000,6.000000,2.000000,2.000000,4.000000,3.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,3.000000,3.000000,7.000000,3.000000,7.000000,3.000000,2.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,15.000000,2.000000,2.000000,8.000000,3.000000,3.000000,3.000000,4.000000,2.000000,6.000000,4.000000,4.000000,3.000000,2.000000,4.000000,10.000000
4,ConvNextV2-Base,1.000000,1.000000,1.000000,11.000000,2.000000,1.000000,2.000000,5.000000,1.000000,1.000000,1.000000,3.000000,1.000000,4.000000,1.000000,1.000000,6.000000,1.000000,7.000000,1.000000,1.000000,1.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000,1.000000,1.000000,6.000000,1.000000,4.000000,2.000000,1.000000,5.000000,3.000000,4.000000,4.000000,3.000000,12.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000,10.000000,7.000000,8.000000,1.000000,2.000000,

In [7]:
from scipy.stats import spearmanr

def reorder_columns_by_similarity(df):
    """
    Reorder the columns of a DataFrame based on their similarity to the second column,
    while keeping the first column in place.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.

    Returns:
    pandas.DataFrame: The DataFrame with reordered columns.
    """
    # Get the first and second column names
    first_column = df.columns[0]
    second_column = df.columns[1]

    # Calculate the Spearman rank correlation between the second column and the others
    correlations = {column: spearmanr(df[second_column], df[column]).correlation for column in df.columns if column != first_column}

    # Sort the columns by their correlation to the second column in descending order
    sorted_columns = sorted(correlations, key=correlations.get, reverse=True)

    # Keep the first column in its place and reorder the rest
    df = df[[first_column] + sorted_columns]

    return df

df = reorder_columns_by_similarity(df)


In [8]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

,Model.Metric,imagenet1k.acc@5,imagenet1k.loss,imagenet1k.acc@1,pascal.mIoU,places365.loss,places365.acc@5,cifar100.loss,hmdb51.acc@1,kinetics.acc@5,food101.loss,places365.acc@1,cifar100.acc@5,food101.acc@5,kinetics.acc@1,diabetic.auc,aircraft.acc@1,cubirds.acc@1,flickr30k.image-to-text-acc@5,flickr30k.text-to-image-acc@5,food101.acc@1,newyorkercaptioncontest.text-to-image-acc@5,flickr30k.image-to-text-acc@1,vgg.acc@1,fungi.acc@1,newyorkercaptioncontest.image-to-text-acc@5,flickr30k.text-to-image-acc@1,newyorkercaptioncontest.text-to-image-acc@1,newyorkercaptioncontest.image-to-text-acc@1,mini.acc@1,clevr-math.acc@1,ucf.acc@1,ham10k.auc,coco-10k.mIoU,dtextures.acc@1,coco-164k.mIoU,iwildcam.mae,ade20k.mIoU,omniglot.acc@1,clevr-math.acc@5,cifar100.acc@1,ucf.acc@5,chexpert.auc,cityscapes.mIoU,acdc.dice-loss,pokemonblipcaptions.text-to-image-acc@5,pokemonblipcaptions.image-to-text-acc@5,cityscapes.dice-loss,clevr.acc@1,pokemonblipcaptions.image-to-text-acc@1,pokemonblipcaptions.text-to-image-acc@1,coco-164k.dice-loss,nyu.mIoU,coco-10k.dice-loss,winoground.text-to-image-acc@1,winoground.image-to-text-acc@1,pascal.dice-loss,nyu.dice-loss,ade20k.dice-loss,hmdb51.acc@5
0,AR-ViT-B16-224,8.000000,5.000000,5.000000,9.000000,10.000000,10.000000,10.000000,7.000000,8.000000,8.000000,10.000000,10.000000,10.000000,7.000000,9.000000,8.000000,8.000000,7.000000,8.000000,10.000000,8.000000,7.000000,8.000000,9.000000,8.000000,4.000000,7.000000,6.000000,3.000000,8.000000,6.000000,6.000000,9.000000,6.000000,9.000000,13.000000,11.000000,6.000000,8.000000,11.000000,5.000000,10.000000,11.000000,7.000000,6.000000,6.000000,16.000000,2.000000,5.000000,6.000000,12.000000,1.000000,16.000000,1.000000,1.000000,16.000000,11.000000,15.000000,11.000000
1,BART,15.000000,15.000000,15.000000,13.000000,15.000000,15.000000,15.000000,12.000000,13.000000,15.000000,15.000000,16.000000,15.000000,13.000000,15.000000,15.000000,13.000000,16.000000,15.000000,16.000000,15.000000,15.000000,14.000000,13.000000,15.000000,14.000000,14.000000,14.000000,14.000000,15.000000,15.000000,14.000000,13.000000,15.000000,13.000000,14.000000,13.000000,15.000000,15.000000,2.000000,13.000000,15.000000,13.000000,14.000000,13.000000,12.000000,9.000000,13.000000,13.000000,13.000000,9.000000,14.000000,7.000000,14.000000,9.000000,4.000000,4.000000,3.000000,1.000000
2,BERT,14.000000,14.000000,14.000000,14.000000,13.000000,13.000000,13.000000,12.000000,15.000000,13.000000,12.000000,14.000000,13.000000,16.000000,13.000000,13.000000,15.000000,14.000000,14.000000,14.000000,14.000000,15.000000,14.000000,14.000000,14.000000,14.000000,16.000000,14.000000,13.000000,12.000000,13.000000,12.000000,15.000000,14.000000,15.000000,15.000000,15.000000,14.000000,13.000000,14.000000,11.000000,13.000000,13.000000,16.000000,11.000000,11.000000,11.000000,12.000000,10.000000,12.000000,10.000000,10.000000,8.000000,7.000000,5.000000,3.000000,5.000000,4.000000,3.000000
3,CLIP-B16-224,2.000000,2.000000,4.000000,2.000000,3.000000,3.000000,5.000000,3.000000,3.000000,3.000000,3.000000,5.000000,4.000000,2.000000,2.000000,3.000000,2.000000,2.000000,3.000000,3.000000,5.000000,2.000000,2.000000,3.000000,5.000000,2.000000,5.000000,5.000000,4.000000,4.000000,3.000000,3.000000,4.000000,4.000000,2.000000,7.000000,3.000000,2.000000,5.000000,6.000000,4.000000,8.000000,9.000000,4.000000,4.000000,4.000000,7.000000,3.000000,6.000000,2.000000,6.000000,5.000000,11.000000,4.000000,10.000000,8.000000,15.000000,10.000000,7.000000
4,ConvNextV2-Base,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,3.000000,2.000000,2.000000,3.000000,3.000000,1.000000,1.000000,3.000000,2.000000,4.000000,4.000000,5.000000,1.000000,2.000000,1.000000,6.000000,3.000000,7.000000,4.000000,2.000000,6.000000,1.000000,1.000000,1.000000,5.000000,3.000000,11.000000,8.000000,8.000000,1.000000,4.000000,7.000000,10.000000,1.000000,12.000000,1.000000,13.000000,15.000000,1.00000

In [16]:
import plotly.graph_objects as go
import pandas as pd

def plot_heatmap(df: pd.DataFrame):
    """
    Plot a heatmap of the DataFrame using Plotly, with rows sorted by their average rank.
    The x-axis labels reflect the order of the columns after they have been reordered by similarity.

    Parameters:
    df (pandas.DataFrame): The DataFrame to plot.

    Returns:
    None
    """
    # Exclude the label column from the data to be plotted
    labels = df[df.columns[0]]
    data = df[df.columns[1:]]

    # Add a temporary column to the DataFrame with the average rank of each row
    data['average_rank'] = data.mean(axis=1)

    # Sort the DataFrame by this temporary column
    data.sort_values('average_rank', ascending=False, inplace=True)

    # Remove the temporary column
    data.drop('average_rank', axis=1, inplace=True)

    # Update the labels to match the new order
    labels = labels[data.index]
    data.columns = [column.replace("flickr30k", "f30k").replace("winoground", "wg").replace("pokemonblipcaptions", "pbc").replace("newyorkercaptioncontest", "nycc").replace("image-to-text", "img-to-txt").replace("text-to-image", "txt-to-img") for column in data.columns]
    
    fig = go.Figure(data=go.Heatmap(
        z=data.values,
        x=data.columns,
        y=labels,
        colorscale='inferno',  # Use reversed Red color scale
    ))

    fig.update_layout(
        title='Model vs Dataset Rankings',
        xaxis_nticks=len(data.columns),
        yaxis_nticks=len(labels),
        autosize=True,
        width=800,
        # height=600,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        )
    )

    fig.show()
    fig.write_image("heatmap.png", scale=5)

plot_heatmap(df)


In [36]:
import pandas as pd
import numpy as np

def process_csv(input_file, output_file, task_mapping, model_name_column_id="Model.Metric"):
    # Load the data
    df = pd.read_csv(input_file)
    
        # Concatenating the header with the first row
    new_headers = [f"{col.split('.')[0]}.{df.iloc[0][idx]}" for idx, col in enumerate(df.columns)]

    # Setting the new concatenated values as column names
    df.columns = new_headers

    # Removing the first row from the DataFrame
    df = df.drop(df.index[0])

    # Resetting the DataFrame index
    df.reset_index(drop=True, inplace=True)
    # Initialize an empty list to hold our reshaped data
    reshaped_data = []
    
    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        model_name = row[model_name_column_id]
        for metric in row.index[1:]:  # Skip the 'Model' column
            value = row[metric]
            
            reshaped_data.append([metric, model_name, value])
    
    # Convert the reshaped data into a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data, columns=['Metric', 'Model', 'Value'])
    
    # Pivot the DataFrame
    pivot_df = reshaped_df.pivot(index='Metric', columns='Model', values='Value').reset_index()
    
    # Add a 'Task Type' column based on the task_mapping
    pivot_df['Task Type'] = pivot_df['Metric'].apply(lambda x: next((task for task, metrics in task_mapping.items() if x.replace(',', '/') in metrics), 'Unknown'))

    # Reorder the DataFrame based on the 'Task Type' and 'Metric'
    pivot_df = pivot_df.sort_values(by=['Task Type', 'Metric'])

    # Move 'Task Type' and 'Metric' to the first columns
    cols = pivot_df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    pivot_df = pivot_df[cols]
    
    cols = ['Task Type', 'Metric'] + [col for col in pivot_df if (col != 'Task Type' and col != 'Metric')]
    pivot_df = pivot_df[cols]
    
    df = pivot_df
        # Create an empty DataFrame with the same columns for the output
    output_data = []

    # Track the last task type encountered
    last_task_type = None

    for index, row in df.iterrows():
        # Check if the current row's task type is different from the last one processed
        if row['Task Type'] != last_task_type:
            # Insert a divider row for the new task type
            divider_row = pd.Series(index=df.columns)
            divider_row['Metric'] = row['Task Type']  # Place the task type in the 'Metric' column
            output_data.append(divider_row)
            
            last_task_type = row['Task Type']  # Update the last task type encountered

        # Append the current row to the output DataFrame
        output_data.append(row)
    output_df = pd.DataFrame(output_data, columns=df.columns)
    # Optionally, you might want to clean or process the 'Task Type' column or rows as needed for your output
    # For instance, you can drop the 'Task Type' column if it's no longer needed:
    output_df.drop(columns=['Task Type'], inplace=True, errors='ignore')

    # Save the reformatted DataFrame to a new CSV file
    output_df.to_csv('formatted_output.csv', index=False)

    # Save the reshaped DataFrame to a new CSV file
    pivot_df.to_csv(output_file, index=False)
# Define the task mapping
task_mapping = {
    "Image Classification": ["cifar100.loss", "cifar100.acc@1", "cifar100.acc@5", "food101.loss", "food101.acc@5", "food101.acc@1", 
                             "imagenet1k.loss", "imagenet1k.acc@5", "imagenet1k.acc@1", "places365.loss", "places365.acc@5", "places365.acc@1"],
    "Few Shot Image Classification": ["fungi.acc@1", "mini.acc@1", "omniglot.acc@1", "vgg.acc@1", "aircraft.acc@1", "dtextures.acc@1", "cubirds.acc@1"],
    "Image Segmentation": ["ade20k.dice-loss", "ade20k.mIoU", "coco-10k.mIoU", "coco-10k.dice-loss", "coco-164k.mIoU", "coco-164k.dice-loss", 
                           "nyu.mIoU", "nyu.dice-loss", "pascal.mIoU", "pascal.dice-loss"],
    "Image Relational Reasoning": ["clevr.acc@1", "clevr-math.acc@5", "clevr-math.acc@1"],
    "Image to Text Zero Shot Learning": ["flickr30k.text-to-image-acc@5", "flickr30k.text-to-image-acc@1", "flickr30k.image-to-text-acc@1", "flickr30k.image-to-text-acc@5",
                                         "newyorkercaptioncontest.text-to-image-acc@5", "newyorkercaptioncontest.text-to-image-acc@1", 
                                         "newyorkercaptioncontest.image-to-text-acc@1", "newyorkercaptioncontest.image-to-text-acc@5",
                                         "pokemonblipcaptions.text-to-image-acc@5", "pokemonblipcaptions.text-to-image-acc@1", 
                                         "pokemonblipcaptions.image-to-text-acc@1", "pokemonblipcaptions.image-to-text-acc@5"],
    "Medical Classification/Segmentation": ["ham10k.auc", "chexpert.auc", "diabetic.auc", "acdc.dice-loss"],
    "Video Classification": ["kinetics.acc@5", "kinetics.acc@1", "hmdb51.acc@5", "hmdb51.acc@1", "ucf.acc@5", "ucf.acc@1"],
    "Video Regression": ["iwildcam.mae"]
}


# Call the function with your input and output file paths and the task mapping
process_csv('03032024-full.csv', 'test1.csv', task_mapping)


/tmp/ipykernel_14685/2591777449.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_headers = [f"{col.split('.')[0]}.{df.iloc[0][idx]}" for idx, col in enumerate(df.columns)]
/tmp/ipykernel_14685/2591777449.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Few Shot Image Classification' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  divider_row['Metric'] = row['Task Type']  # Place the task type in the 'Metric' column
/tmp/ipykernel_14685/2591777449.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Image Classification' has dtype incompatible with float64, please explicitly cast to a c